# IMPORTS

In [1]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal
import pickle
from pathlib import Path

# SETUP

In [2]:
dir_tree_util_path = os.path.join("utils", "dir_tree.py")
exec(open(dir_tree_util_path).read())

# INPUTS

In [4]:
# Paths
path_input = PROJECT_DIRS["DADOS_VEQ_ANTT_DIR"]
path_output = PROJECT_DIRS["DADOS_DERIVADOS_DIR"]

In [5]:
periodo = list(range(2010, 2025))

In [69]:
periodos_CAGR = [[2010,2023],[2016,2023]]

# Criando um dataframe consolidado com os dados de tráfego

In [6]:
# arquivos dos dados
arquivos = os.listdir(path_input)

In [7]:
df_trafego = pd.DataFrame()
for ano in periodo:
    arquivo = [a for a in arquivos if a[:-4].endswith(str(ano))][0]
    df_ano = pd.read_csv(os.path.join(path_input, arquivo), sep=';', encoding='cp1252', low_memory=False)
    df_trafego = pd.concat([df_trafego,df_ano])

# EDA Inicial

In [8]:
df_trafego.head()

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
0,RODOVIA DO AÇO,01-01-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,44146,1,44146,NaN
1,RODOVIA DO AÇO,01-01-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,35771,1,35771,NaN
2,RODOVIA DO AÇO,01-02-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,33455,1,33455,NaN
3,RODOVIA DO AÇO,01-02-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,27109,1,27109,NaN
4,RODOVIA DO AÇO,01-03-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,31677,1,31677,NaN


In [9]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 10 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   concessionaria              object
 1   mes_ano                     object
 2   sentido                     object
 3   praca                       object
 4   categoria                   object
 5   tipo_de_veiculo             object
 6   volume_total                object
 7   multiplicador_de_tarifa     object
 8   volume_veiculo_equivalente  object
 9   tipo_de_cobranca            object
dtypes: object(10)
memory usage: 251.8+ MB


In [10]:
for col in df_trafego:
    print(f"{col}: {df_trafego[col].isna().sum()}")

concessionaria: 0
mes_ano: 0
sentido: 0
praca: 0
categoria: 0
tipo_de_veiculo: 0
volume_total: 344
multiplicador_de_tarifa: 0
volume_veiculo_equivalente: 0
tipo_de_cobranca: 74780


In [11]:
df_trafego[df_trafego['volume_total'].isna()].sample(9)

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
125942,ECOSUL,01/10/2023,Crescente,"Praça 03 BR-116/RS km 541,20",Categoria 9,Moto,NaN,"0,50","0,00",N/I
125912,ECOSUL,01/10/2023,Decrescente,"Praça 01 BR-116/RS km 430,79",Categoria 9,Moto,NaN,"0,50","0,00",N/I
123011,ECOPONTE,01/07/2023,Decrescente,"Praça 01 BR-101/RJ km 322,20",Veículo Comercial Acima 10 eixos,COMERCIAL,NaN,"10,00","0,00",N/I
125472,ECOSUL,01/05/2023,Decrescente,"Praça 04 BR-392/RS km 52,30",Categoria 9,Moto,NaN,"0,50","0,00",N/I
126032,ECOSUL,01/11/2023,Decrescente,"Praça 02 BR-116/RS km 510,76",Categoria 9,Moto,NaN,"0,50","0,00",N/I
121118,ECO050,01/12/2023,Crescente,"Praça 02 BR-050/GO km 226,00",Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
126162,ECOSUL,01/12/2023,Crescente,"Praça 04 BR-392/RS km 52,30",Categoria 9,Moto,NaN,"0,50","0,00",N/I
134821,RIOSP,01/03/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 7,Comercial,NaN,"5,00","0,00",N/I
125202,ECOSUL,01/03/2023,Crescente,"Praça 01 BR-116/RS km 430,79",Categoria 9,Moto,NaN,"0,50","0,00",N/I


In [12]:
df_trafego[df_trafego['volume_total'].isna()]['volume_veiculo_equivalente'].unique()

array(['0,00'], dtype=object)

In [13]:
df_trafego['mes_ano'].str[:2].unique()

array(['01', '14', '28', '31', '30', '02', '03', '04', '05', '06', '07',
       '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27', '29'], dtype=object)

In [14]:
df_trafego['mes_ano'].str[3:5].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12'], dtype=object)

In [15]:
df_trafego['tipo_de_cobranca'].unique()

array([nan, 'N/I', 'Manual', 'Automática', 'Mista'], dtype=object)

In [16]:
df_trafego['volume_total'].isna().any()

np.True_

In [17]:
df_trafego['volume_total'].isna().sum()

np.int64(344)

In [18]:
df_trafego['multiplicador_de_tarifa'].unique()

array(['1', '2', '1,5', '3', '4', '5', '6', '0,5', '7', '8', '9', '10',
       1.0, 2.0, 1.5, 3.0, 4.0, 5.0, 6.0, 0.5, 7.0, 8.0, 9.0, 10.0,
       '2,00', '3,00', '4,00', '5,00', '6,00', '0,50', '1,00', '1,50',
       '7,00', '8,00', '9,00', '10,00', '0,00', '11,00', '13,00', '15,00',
       '12,00', '14,00', '18,00', '20,00', '16,00', '17,00', '19,00'],
      dtype=object)

# Tratamento dos dados

In [19]:
# Colunas de ano, mes e dia
df_trafego['year'] = df_trafego['mes_ano'].str[-4:].astype('int') 
df_trafego['month'] = df_trafego['mes_ano'].str[3:5].astype('int')
df_trafego['day'] = df_trafego['mes_ano'].str[:2].astype('int') 

In [20]:
# Recriando a coluna de mes_ano, agora como datetime ao invés de string. Tem que ser em ingles pq a função do pandas exige
df_trafego['mes_ano'] = pd.to_datetime(df_trafego[['year', 'month', 'day']]).dt.date

In [21]:
# preenchendo os NaNs, qdo aplicável:
df_trafego['volume_total'] = df_trafego['volume_total'].fillna(0) #o volume equivalente é zero sempre que volume_total é zero

In [22]:
# Convertendo os tipos de dados para numéricos:
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.split('.')[0] if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].astype('int')

df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].astype('float') # existe multiplicador de tarifa fracionário

In [23]:
# convertendo categoria para string (do contrário não salva para parquet):
df_trafego['categoria'] = df_trafego['categoria'].astype('string')

In [24]:
# convertendo o multiplicador de tarifa de string para decimal (p/manter a precisao):
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(Decimal)

In [25]:
# O tipo de tráfego ora está em maisúcula, ora em minúscula.
# Colocando tudo p/maiúscula
df_trafego['tipo_de_veiculo'] = df_trafego['tipo_de_veiculo'].str.upper()

In [26]:
# algumas concessionárias estão ora em maiúsculas ora em minúsculas
# colocando tudo para maiúscula
df_trafego['concessionaria'] = df_trafego['concessionaria'].str.upper()

In [27]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 13 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   concessionaria              object 
 1   mes_ano                     object 
 2   sentido                     object 
 3   praca                       object 
 4   categoria                   string 
 5   tipo_de_veiculo             object 
 6   volume_total                int64  
 7   multiplicador_de_tarifa     object 
 8   volume_veiculo_equivalente  float64
 9   tipo_de_cobranca            object 
 10  year                        int64  
 11  month                       int64  
 12  day                         int64  
dtypes: float64(1), int64(4), object(7), string(1)
memory usage: 320.5+ MB


In [28]:
df_trafego['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024])

In [29]:
df_trafego.query("year == 2024")['month'].unique()

array([1, 2, 3, 4, 5, 6])

## Criando coluna desambiguando categorias comercial x passeio

In [30]:
df_trafego['tipo_de_veiculo'].unique()

array(['PASSEIO', 'COMERCIAL', 'MOTO', 'VEÍCULO PEQUENO'], dtype=object)

In [31]:
df_trafego["TIPO_TRAFEGO"] = df_trafego['tipo_de_veiculo'].apply(lambda x: x if x == "COMERCIAL" else "PASSEIO")

In [32]:
df_trafego["TIPO_TRAFEGO"].unique()

array(['PASSEIO', 'COMERCIAL'], dtype=object)

## Criando colunas com trafégo em Veqs para comercial e passeio

In [33]:
df_trafego = df_trafego.rename(columns={'volume_veiculo_equivalente':'VEQS_TOTAL'})
df_trafego['VEQS_COMERCIAL'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'COMERCIAL',
    df_trafego['VEQS_TOTAL'],
    0
)
df_trafego['VEQS_PASSEIO'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'PASSEIO',
    df_trafego['VEQS_TOTAL'],
    0
)

# Criando dataframe com o período inicial e final dos dados por concessionária

In [34]:
df_periodos = df_trafego.groupby('concessionaria').agg(
    data_inicial=('mes_ano', 'min'),
    data_final=('mes_ano', 'max')
).reset_index()
df_periodos['data_inicial'] = pd.to_datetime(df_periodos['data_inicial']).dt.date
df_periodos['data_final'] = pd.to_datetime(df_periodos['data_final']).dt.date

In [35]:
df_periodos = df_periodos.sort_values(by='data_inicial', ascending=False)

# Criando dataframes agrupando VEQs por ano/concessionaria/tipo de trafego

In [57]:
df_trafego.columns

Index(['concessionaria', 'mes_ano', 'sentido', 'praca', 'categoria',
       'tipo_de_veiculo', 'volume_total', 'multiplicador_de_tarifa',
       'VEQS_TOTAL', 'tipo_de_cobranca', 'year', 'month', 'day',
       'TIPO_TRAFEGO', 'VEQS_COMERCIAL', 'VEQS_PASSEIO'],
      dtype='object')

In [58]:
cols_veqs = ['VEQS_TOTAL','VEQS_COMERCIAL','VEQS_PASSEIO']

In [59]:
dict_veqs = {}
for col in cols_veqs:
    dict_veqs[col] = df_trafego.pivot_table(
        index='concessionaria',
        columns='year',
        values=col,
        aggfunc='sum'
        )

In [52]:
dict_veqs['VEQS_TOTAL']

year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
concessionaria,,,,,,,,,,,,,,,
AUTOPISTA FERNÃO DIAS,119092267.5,150559736.0,157806037.0,162107028.0,161068092.0,148094701.5,142458018.5,147170102.5,147290900.0,169882815.5,168401716.5,1.896789e+08,162456829.0,1.679139e+08,12264573.0
AUTOPISTA FLUMINENSE,41728899.5,45660152.0,47804013.5,47951191.5,48207454.0,45600603.0,41249860.5,39885704.5,39286923.0,40566531.5,38990241.5,4.459816e+07,44950482.0,4.660152e+07,9470489.5
AUTOPISTA LITORAL SUL,107872122.5,112844729.5,120340610.0,119069671.5,139077138.5,129296415.5,123865581.0,121079477.0,122937552.5,129407313.0,126919283.0,1.401476e+08,140484365.0,1.485568e+08,45936133.5
AUTOPISTA PLANALTO SUL,25946947.5,27122408.5,27055445.0,28234697.0,29643995.5,26063140.5,25035859.0,24908641.5,26075591.0,27242211.5,27119423.5,3.088398e+07,32397859.0,3.311986e+07,11766091.0
AUTOPISTA REGIS BITTENCOURT,132761912.5,141009043.5,143187465.0,146902608.5,144880653.0,130744094.0,124006297.0,129737477.5,133263422.5,138980100.0,139187866.5,1.550483e+08,153731701.0,1.569003e+08,13948155.5
CONCEBRA,NaN,NaN,NaN,NaN,NaN,44784896.0,83469296.5,83991534.5,82802394.5,85876376.0,85486882.0,9.253040e+07,90947633.0,9.367429e+07,46032729.0
CONCEPA,29641914.5,32032526.0,34632455.5,36922811.0,39101463.5,37236391.0,35607856.5,37787017.5,20508556.0,NaN,NaN,NaN,NaN,NaN,NaN
CONCER,25050336.0,27478441.5,29282042.5,30236236.5,29587870.5,25994928.5,23552236.0,22784899.5,22615413.5,23558604.0,20635092.0,2.292358e+07,23483295.5,2.484375e+07,10319138.5
CRO,NaN,NaN,NaN,NaN,NaN,29519863.0,86930106.0,85297258.0,91150072.0,92706338.0,95197099.0,1.040709e+08,109077798.5,1.150608e+08,40578829.5


# CAGR

In [52]:
def filter_periodos(df_periodos, periodo: list):
    # Convert data_inicial and data_final to consistent date format
    df_periodos['data_inicial'] = pd.to_datetime(df_periodos['data_inicial']).dt.date
    df_periodos['data_final'] = pd.to_datetime(df_periodos['data_final']).dt.date

    # Define start and end dates
    start_date = pd.to_datetime(f'{periodo[0]}-01-01').date()
    end_date = pd.to_datetime(f'{periodo[1]}-12-31').date()

    # Apply the mask: Match start date exactly and ensure the period ends on or after the specified end date
    mask = (df_periodos['data_inicial'] == start_date) & (df_periodos['data_final'] >= end_date)

    # Filter the dataframe
    df_filtrada_periodo = df_periodos[mask]
    
    return df_filtrada_periodo

In [61]:
def calc_cagr(dict_veqs:dict, df_periodos, periodos:list):
    # Criando dataframes para os cálculos e condições
    df_CAGR = pd.DataFrame(index=df_periodos.index)
    df_periodos.index = df_periodos['concessionaria']
    
    for key, df_veqs in dict_veqs.items():
        
        if 'data_inicial' not in df_veqs.columns and 'data_final' not in df_veqs.columns:
            df_veqs = df_veqs.join(df_periodos[['data_inicial','data_final']])
        # df_veqs = df_veqs.join(df_periodos[['data_inicial','data_final']])
    
        for anos in periodos:
            # Condição para calcular apenas para anos completos no período
            start_date = pd.to_datetime(f'{anos[0]}-01-01').date()
            end_date = pd.to_datetime(f'{anos[1]}-12-31').date()
            # condition = (df_CAGR['data_inicial'] <= start_date) & (df_CAGR['data_final'] >= end_date)
            condition = (df_veqs['data_inicial'] <= start_date) & (df_veqs['data_final'] >= end_date)

            # Calculando o CAGR no período
            col_CAGR = f'CAGR{anos[0]}-{anos[1]}_%'      
            df_veqs[col_CAGR] = np.where(condition,
                                        ( ( (df_veqs[anos[1]] / df_veqs[anos[0]]) ** (1 / (anos[1]-anos[0])) ) - 1 ) * 100,
                                         'N/D'
                                        )
        dict_veqs[key] = df_veqs
        
    return dict_veqs

In [70]:
dict_veqs = calc_cagr(dict_veqs, df_periodos, periodos_CAGR)

# Salvando p/parquet

In [65]:
# dataframe total
# df_trafego.to_parquet(os.path.join(path_dados_derivados,'df_trafego.parquet'))
df_trafego.to_parquet(path_output /'df_trafego.parquet')

In [66]:
# data inicial e final
df_periodos.to_parquet(path_output /'df_periodos.parquet')

In [67]:
# dicionário com os dados de tráfego:
for veq in dict_veqs:
    dict_veqs[veq].to_parquet(path_output / f'df_{veq}.parquet')

/home/andcm/.cache/pypoetry/virtualenvs/infra-data-J9p6eQia-py3.9/lib/python3.9/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


# Salvando p/Excel

In [68]:
with pd.ExcelWriter(path_output / 'dados_trafego_ANTT.xlsx') as writer:
    df_periodos.to_excel(writer, sheet_name="periodo_concessionarias")
    for veq in dict_veqs:
        dict_veqs[veq].to_excel(writer, sheet_name=f"{veq}")